<a href="https://colab.research.google.com/github/alexandergribenchenko/Data_Science_Toolkit/blob/main/Exchange/AV_Test_Entrega.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

# Filtrar y silenciar todos los warnings
warnings.filterwarnings("ignore")

In [2]:
path_github = 'https://raw.githubusercontent.com/alexandergribenchenko/Data_Science_Toolkit/main/Exchange/entrega_sample.json'

In [3]:
import pandas as pd

In [4]:
df = pd.read_json(path_github, orient='records', lines=True, dtype='object')
df = df.sort_values(by = ['airline_code', 'rt', 'flight_date', 'snapshot_date'],
                                  ascending= [True, True, False, True])
df

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks
1576,20220213BZECUK2M202201,20220213,202201,BZECUK,CAMCAM,OTHS,2M,103272.0
83,20220227BZECUK2M202201,20220227,202201,BZECUK,CAMCAM,OTHS,2M,103272.0
86,20220227BZECYC2M202203,20220227,202203,BZECYC,CAMCAM,OTHS,2M,322.0
1014,20220313BZECYC2M202203,20220313,202203,BZECYC,CAMCAM,OTHS,2M,322.0
2141,20220327BZECYC2M202203,20220327,202203,BZECYC,CAMCAM,OTHS,2M,322.0
...,...,...,...,...,...,...,...,...
1280,20230319OAKSALY4202303,20230319,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0
1072,20230326OAKSALY4202303,20230326,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0
1083,20230507BLBUIO~~202305,20230507,202305,BLBUIO,CAMSAM,ECUINT,~~,1606515.0
1098,20230507ETRUIO~~202305,20230507,202305,ETRUIO,DOM_EC,ECUDOM,~~,2457782.0


In [5]:
df_flight_date_max_min = df.groupby(['airline_code', 'rt', 'region', 'hub', 'flight_date'], as_index=False).\
          agg(max_snapshot_date=('snapshot_date', 'max'),
              min_snapshot_date=('snapshot_date', 'min')
              ).\
          sort_values(by=['airline_code', 'rt'], ascending=[True,True])

df_flight_date_max_min

,airline_code,rt,region,hub,flight_date,max_snapshot_date,min_snapshot_date
0,2M,BZECUK,CAMCAM,OTHS,202201,20220227,20220213
1,2M,BZECYC,CAMCAM,OTHS,202203,20220424,20220227
2,2M,BZEDGA,CAMCAM,OTHS,202201,20220227,20220213
3,2M,BZEINB,CAMCAM,OTHS,202201,20220227,20220213
4,2M,BZEPLJ,CAMCAM,OTHS,202201,20220227,20220213
...,...,...,...,...,...,...,...
580,Y2,CTGJBQ,CARALL,EXBOGINT,202208,20220911,20220619
581,Y4,OAKSAL,CAMNAM,SAL,202303,20230326,20221204
582,~~,BLBUIO,CAMSAM,ECUINT,202305,20230507,20230507
583,~~,ETRUIO,DOM_EC,ECUDOM,202305,20230507,20230507


In [6]:
df_flight_date_max_min['max_snapshot_date'] = pd.to_datetime(df_flight_date_max_min['max_snapshot_date'], format='%Y%m%d')
df_flight_date_max_min['min_snapshot_date'] = pd.to_datetime(df_flight_date_max_min['min_snapshot_date'], format='%Y%m%d')

In [7]:
df_flight_date_max_min

,airline_code,rt,region,hub,flight_date,max_snapshot_date,min_snapshot_date
0,2M,BZECUK,CAMCAM,OTHS,202201,2022-02-27,2022-02-13
1,2M,BZECYC,CAMCAM,OTHS,202203,2022-04-24,2022-02-27
2,2M,BZEDGA,CAMCAM,OTHS,202201,2022-02-27,2022-02-13
3,2M,BZEINB,CAMCAM,OTHS,202201,2022-02-27,2022-02-13
4,2M,BZEPLJ,CAMCAM,OTHS,202201,2022-02-27,2022-02-13
...,...,...,...,...,...,...,...
580,Y2,CTGJBQ,CARALL,EXBOGINT,202208,2022-09-11,2022-06-19
581,Y4,OAKSAL,CAMNAM,SAL,202303,2023-03-26,2022-12-04
582,~~,BLBUIO,CAMSAM,ECUINT,202305,2023-05-07,2023-05-07
583,~~,ETRUIO,DOM_EC,ECUDOM,202305,2023-05-07,2023-05-07


In [8]:
dfs_list = []

# Agrupamos el dataframe original por la combinación de 'airline_code', 'rt' y 'flight_date'
grouped = df_flight_date_max_min.groupby(['airline_code', 'rt', 'flight_date'])

# Iteramos sobre cada grupo y generamos las fechas en intervalos semanales entre 'min_snapshot_date' y 'max_snapshot_date'
for _, group_df in grouped:
    date_range = pd.date_range(start=group_df['min_snapshot_date'].iloc[0], end=group_df['max_snapshot_date'].iloc[0], freq='W')
    # Convertimos las fechas a formato de cadena (str) con el formato deseado 'yyyymmdd'
    date_range_str = date_range.strftime('%Y%m%d')
    # Creamos un nuevo dataframe que contenga todas las fechas generadas para este grupo
    new_df = pd.DataFrame({
        'airline_code': group_df['airline_code'].iloc[0],
        'rt': group_df['rt'].iloc[0],
        'flight_date': group_df['flight_date'].iloc[0],
        'snapshot_date': date_range_str
    })
    # Agregamos el nuevo dataframe a la lista de dataframes
    dfs_list.append(new_df)

# Concatenamos todos los dataframes generados en uno solo
result_df = pd.concat(dfs_list, ignore_index=True)

In [9]:
df

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks
1576,20220213BZECUK2M202201,20220213,202201,BZECUK,CAMCAM,OTHS,2M,103272.0
83,20220227BZECUK2M202201,20220227,202201,BZECUK,CAMCAM,OTHS,2M,103272.0
86,20220227BZECYC2M202203,20220227,202203,BZECYC,CAMCAM,OTHS,2M,322.0
1014,20220313BZECYC2M202203,20220313,202203,BZECYC,CAMCAM,OTHS,2M,322.0
2141,20220327BZECYC2M202203,20220327,202203,BZECYC,CAMCAM,OTHS,2M,322.0
...,...,...,...,...,...,...,...,...
1280,20230319OAKSALY4202303,20230319,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0
1072,20230326OAKSALY4202303,20230326,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0
1083,20230507BLBUIO~~202305,20230507,202305,BLBUIO,CAMSAM,ECUINT,~~,1606515.0
1098,20230507ETRUIO~~202305,20230507,202305,ETRUIO,DOM_EC,ECUDOM,~~,2457782.0


In [10]:
result_df.head()

,airline_code,rt,flight_date,snapshot_date
0,2M,BZECUK,202201,20220213
1,2M,BZECUK,202201,20220220
2,2M,BZECUK,202201,20220227
3,2M,BZECYC,202203,20220227
4,2M,BZECYC,202203,20220306


In [11]:
merged_df = pd.merge(df, result_df, on=['airline_code', 'rt', 'flight_date', 'snapshot_date'], how='right')
merged_df = merged_df.sort_values(by = ['airline_code', 'rt', 'flight_date', 'snapshot_date'],
                                  ascending= [True, True, False, True])
merged_df['asks'] = merged_df['asks'].fillna(0)
merged_df

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks
0,20220213BZECUK2M202201,20220213,202201,BZECUK,CAMCAM,OTHS,2M,103272.0
1,NaN,20220220,202201,BZECUK,NaN,NaN,2M,0.0
2,20220227BZECUK2M202201,20220227,202201,BZECUK,CAMCAM,OTHS,2M,103272.0
3,20220227BZECYC2M202203,20220227,202203,BZECYC,CAMCAM,OTHS,2M,322.0
4,NaN,20220306,202203,BZECYC,NaN,NaN,2M,0.0
...,...,...,...,...,...,...,...,...
4080,20230319OAKSALY4202303,20230319,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0
4081,20230326OAKSALY4202303,20230326,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0
4082,20230507BLBUIO~~202305,20230507,202305,BLBUIO,CAMSAM,ECUINT,~~,1606515.0
4083,20230507ETRUIO~~202305,20230507,202305,ETRUIO,DOM_EC,ECUDOM,~~,2457782.0


In [12]:
df_sample = merged_df[(merged_df.flight_date=='202306')&
   (merged_df.airline_code=='VE')&
   (merged_df.rt=='AXMCTG')
]
df_sample

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks
2985,20230430AXMCTGVE202306,20230430,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
2986,20230507AXMCTGVE202306,20230507,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
2987,20230514AXMCTGVE202306,20230514,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,95479.0
2988,20230521AXMCTGVE202306,20230521,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
2989,20230528AXMCTGVE202306,20230528,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
2990,20230604AXMCTGVE202306,20230604,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
2991,NaN,20230611,202306,AXMCTG,NaN,NaN,VE,0.0
2992,20230618AXMCTGVE202306,20230618,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
2993,20230625AXMCTGVE202306,20230625,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
2994,NaN,20230702,202306,AXMCTG,NaN,NaN,VE,0.0


In [13]:
columna = 'asks'
columna_categorias = ['airline_code', 'rt', 'flight_date']

# Función personalizada para contar los valores cero en la ventana móvil
def contar_ceros_ventana(serie):
    return (serie[:] == 0).sum()

# Crear una nueva columna con el conteo de ceros en la ventana móvil por categoría
df_sample['valores'] = df_sample.groupby(columna_categorias, group_keys=False)[columna].apply(lambda x: x.rolling(4).apply(contar_ceros_ventana, raw=True))

# Ajustar la nueva columna para que muestre los valores correctos
df_sample['valores'] = df_sample['valores'].shift(1)
df_sample

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks,valores
2985,20230430AXMCTGVE202306,20230430,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,NaN
2986,20230507AXMCTGVE202306,20230507,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,NaN
2987,20230514AXMCTGVE202306,20230514,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,95479.0,NaN
2988,20230521AXMCTGVE202306,20230521,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,NaN
2989,20230528AXMCTGVE202306,20230528,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,0.0
2990,20230604AXMCTGVE202306,20230604,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,0.0
2991,NaN,20230611,202306,AXMCTG,NaN,NaN,VE,0.0,0.0
2992,20230618AXMCTGVE202306,20230618,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,1.0
2993,20230625AXMCTGVE202306,20230625,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,1.0
2994,NaN,20230702,202306,AXMCTG,NaN,NaN,VE,0.0,1.0


In [14]:
columna = 'asks'
columna_categorias = ['airline_code', 'rt', 'flight_date']

# Función personalizada para contar los valores cero en la ventana móvil
def contar_ceros_ventana(serie):
    return (serie[:] == 0).sum()

# Crear una nueva columna con el conteo de ceros en la ventana móvil por categoría
merged_df['Cuenta_ceros_ventana'] = merged_df.groupby(columna_categorias, group_keys=False)[columna].apply(lambda x: x.rolling(4).apply(contar_ceros_ventana, raw=True))

# Ajustar la nueva columna para que muestre los valores correctos
merged_df['Cuenta_ceros_ventana'] = merged_df['Cuenta_ceros_ventana'].shift(1)

In [15]:
merged_df

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks,Cuenta_ceros_ventana
0,20220213BZECUK2M202201,20220213,202201,BZECUK,CAMCAM,OTHS,2M,103272.0,NaN
1,NaN,20220220,202201,BZECUK,NaN,NaN,2M,0.0,NaN
2,20220227BZECUK2M202201,20220227,202201,BZECUK,CAMCAM,OTHS,2M,103272.0,NaN
3,20220227BZECYC2M202203,20220227,202203,BZECYC,CAMCAM,OTHS,2M,322.0,NaN
4,NaN,20220306,202203,BZECYC,NaN,NaN,2M,0.0,NaN
...,...,...,...,...,...,...,...,...,...
4080,20230319OAKSALY4202303,20230319,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0,0.0
4081,20230326OAKSALY4202303,20230326,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0,0.0
4082,20230507BLBUIO~~202305,20230507,202305,BLBUIO,CAMSAM,ECUINT,~~,1606515.0,0.0
4083,20230507ETRUIO~~202305,20230507,202305,ETRUIO,DOM_EC,ECUDOM,~~,2457782.0,NaN


In [16]:
merged_df.Cuenta_ceros_ventana.value_counts(dropna=False)

NaN    1722
2.0    1073
0.0     779
1.0     315
3.0     196
Name: Cuenta_ceros_ventana, dtype: int64

In [17]:
merged_df[merged_df.Cuenta_ceros_ventana==3].sort_values(by = 'snapshot_date' , ascending = False)

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks,Cuenta_ceros_ventana
968,20230723CUNFRSCC202306,20230723,202306,CUNFRS,CAMCAM,OTHS,CC,18878.0,3.0
1002,20230723LCESAPCC202306,20230723,202306,LCESAP,CAMCAM,OTHS,CC,10718.0,3.0
2997,20230723AXMCTGVE202306,20230723,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,3.0
1837,20230723IAHXPLNK202306,20230723,202306,IAHXPL,CAMNAM,OTHS,NK,6906596.0,3.0
1224,20230723MECPTYCM202306,20230723,202306,MECPTY,CAMSAM,ECUINT,CM,553975.0,3.0
...,...,...,...,...,...,...,...,...,...
1225,20220213MGAPTYCM202201,20220213,202201,MGAPTY,CAMCAM,OTHS,CM,7957022.0,3.0
3493,20220213MDEVVCVH202203,20220213,202203,MDEVVC,DOM_CO,EXBOGDOM,VH,222476.0,3.0
1283,20220213PTYTPACM202201,20220213,202201,PTYTPA,CAMNAM,OTHS,CM,12715852.0,3.0
1818,20220213IAHXPLNK202201,20220213,202201,IAHXPL,CAMNAM,OTHS,NK,17057775.0,3.0


In [18]:
df[(df.flight_date=='202306')&
   (df.airline_code=='VE')&
   (df.rt=='AXMCTG')
]

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks
2711,20230430AXMCTGVE202306,20230430,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
1082,20230507AXMCTGVE202306,20230507,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
2470,20230514AXMCTGVE202306,20230514,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,95479.0
2356,20230521AXMCTGVE202306,20230521,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
2182,20230528AXMCTGVE202306,20230528,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
1184,20230604AXMCTGVE202306,20230604,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
968,20230612AXMCTGVE202306,20230612,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
929,20230618AXMCTGVE202306,20230618,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
2271,20230625AXMCTGVE202306,20230625,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0
647,20230723AXMCTGVE202306,20230723,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0


In [19]:
merged_df[(merged_df.flight_date=='202306')&
   (merged_df.airline_code=='VE')&
   (merged_df.rt=='AXMCTG')
]

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks,Cuenta_ceros_ventana
2985,20230430AXMCTGVE202306,20230430,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,NaN
2986,20230507AXMCTGVE202306,20230507,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,NaN
2987,20230514AXMCTGVE202306,20230514,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,95479.0,NaN
2988,20230521AXMCTGVE202306,20230521,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,NaN
2989,20230528AXMCTGVE202306,20230528,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,0.0
2990,20230604AXMCTGVE202306,20230604,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,0.0
2991,NaN,20230611,202306,AXMCTG,NaN,NaN,VE,0.0,0.0
2992,20230618AXMCTGVE202306,20230618,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,1.0
2993,20230625AXMCTGVE202306,20230625,202306,AXMCTG,DOM_CO,EXBOGDOM,VE,63653.0,1.0
2994,NaN,20230702,202306,AXMCTG,NaN,NaN,VE,0.0,1.0


In [26]:
df_01 = merged_df[(merged_df.Cuenta_ceros_ventana==3)&
          (merged_df.id_alert.notna())].reset_index(drop=True)
df_01

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks,Cuenta_ceros_ventana
0,20220717BZEPND2M202206,20220717,202206,BZEPND,CAMCAM,OTHS,2M,1674.0,3.0
1,20220213BZESPR2M202201,20220213,202201,BZESPR,CAMCAM,OTHS,2M,673031.0,3.0
2,20220717GYEMAD2W202211,20220717,202211,GYEMAD,EURALL,ECUINT,2W,7786650.0,3.0
3,20220731GYEMAD2W202211,20220731,202211,GYEMAD,EURALL,ECUINT,2W,7786650.0,3.0
4,20220717AAZGUA5U202206,20220717,202206,AAZGUA,CAMCAM,OTHS,5U,19698.0,3.0
...,...,...,...,...,...,...,...,...,...
121,20220731RTBYYZWS202212,20220731,202212,RTBYYZ,CAMNAM,OTHS,WS,1638286.0,3.0
122,20220717BAQJBQY2202208,20220717,202208,BAQJBQ,CARALL,EXBOGINT,Y2,794372.0,3.0
123,20220731BAQJBQY2202208,20220731,202208,BAQJBQ,CARALL,EXBOGINT,Y2,794372.0,3.0
124,20220717CTGJBQY2202208,20220717,202208,CTGJBQ,CARALL,EXBOGINT,Y2,862608.0,3.0


In [29]:
df_flight_date_min= df.groupby(['airline_code', 'rt', 'region', 'hub', 'flight_date'], as_index=False).\
          agg(snapshot_date=('snapshot_date', 'min')
              ).\
          sort_values(by=['airline_code', 'rt'], ascending=[True,True])

df_flight_date_min['minimo'] = 'minimo'

df_flight_date_min

,airline_code,rt,region,hub,flight_date,snapshot_date,minimo
0,2M,BZECUK,CAMCAM,OTHS,202201,20220213,minimo
1,2M,BZECYC,CAMCAM,OTHS,202203,20220227,minimo
2,2M,BZEDGA,CAMCAM,OTHS,202201,20220213,minimo
3,2M,BZEINB,CAMCAM,OTHS,202201,20220213,minimo
4,2M,BZEPLJ,CAMCAM,OTHS,202201,20220213,minimo
...,...,...,...,...,...,...,...
580,Y2,CTGJBQ,CARALL,EXBOGINT,202208,20220619,minimo
581,Y4,OAKSAL,CAMNAM,SAL,202303,20221204,minimo
582,~~,BLBUIO,CAMSAM,ECUINT,202305,20230507,minimo
583,~~,ETRUIO,DOM_EC,ECUDOM,202305,20230507,minimo


In [30]:
df

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks
1576,20220213BZECUK2M202201,20220213,202201,BZECUK,CAMCAM,OTHS,2M,103272.0
83,20220227BZECUK2M202201,20220227,202201,BZECUK,CAMCAM,OTHS,2M,103272.0
86,20220227BZECYC2M202203,20220227,202203,BZECYC,CAMCAM,OTHS,2M,322.0
1014,20220313BZECYC2M202203,20220313,202203,BZECYC,CAMCAM,OTHS,2M,322.0
2141,20220327BZECYC2M202203,20220327,202203,BZECYC,CAMCAM,OTHS,2M,322.0
...,...,...,...,...,...,...,...,...
1280,20230319OAKSALY4202303,20230319,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0
1072,20230326OAKSALY4202303,20230326,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0
1083,20230507BLBUIO~~202305,20230507,202305,BLBUIO,CAMSAM,ECUINT,~~,1606515.0
1098,20230507ETRUIO~~202305,20230507,202305,ETRUIO,DOM_EC,ECUDOM,~~,2457782.0


In [50]:
salida_01 = pd.merge(df, df_flight_date_min, on=['airline_code', 'rt', 'region', 'hub','flight_date', 'snapshot_date'], how='left')
salida_01 = salida_01.sort_values(by = ['airline_code', 'rt', 'flight_date', 'snapshot_date'],
                                  ascending= [True, True, False, True])
# salida_01['asks'] = salida_01['asks'].fillna(0)
salida_01

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks,minimo
0,20220213BZECUK2M202201,20220213,202201,BZECUK,CAMCAM,OTHS,2M,103272.0,minimo
1,20220227BZECUK2M202201,20220227,202201,BZECUK,CAMCAM,OTHS,2M,103272.0,NaN
2,20220227BZECYC2M202203,20220227,202203,BZECYC,CAMCAM,OTHS,2M,322.0,minimo
3,20220313BZECYC2M202203,20220313,202203,BZECYC,CAMCAM,OTHS,2M,322.0,NaN
4,20220327BZECYC2M202203,20220327,202203,BZECYC,CAMCAM,OTHS,2M,322.0,NaN
...,...,...,...,...,...,...,...,...,...
2916,20230319OAKSALY4202303,20230319,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0,NaN
2917,20230326OAKSALY4202303,20230326,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0,NaN
2918,20230507BLBUIO~~202305,20230507,202305,BLBUIO,CAMSAM,ECUINT,~~,1606515.0,minimo
2919,20230507ETRUIO~~202305,20230507,202305,ETRUIO,DOM_EC,ECUDOM,~~,2457782.0,minimo


In [51]:
salida_02 = pd.merge(salida_01, df_01, on=['id_alert',	'snapshot_date',	'flight_date',	'rt',	'region',	'hub','airline_code',	'asks'], how='left')
salida_02 = salida_02.sort_values(by = ['airline_code', 'rt', 'flight_date', 'snapshot_date'],
                                  ascending= [True, True, False, True])
# salida_01['asks'] = salida_01['asks'].fillna(0)
salida_02

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks,minimo,Cuenta_ceros_ventana
0,20220213BZECUK2M202201,20220213,202201,BZECUK,CAMCAM,OTHS,2M,103272.0,minimo,NaN
1,20220227BZECUK2M202201,20220227,202201,BZECUK,CAMCAM,OTHS,2M,103272.0,NaN,NaN
2,20220227BZECYC2M202203,20220227,202203,BZECYC,CAMCAM,OTHS,2M,322.0,minimo,NaN
3,20220313BZECYC2M202203,20220313,202203,BZECYC,CAMCAM,OTHS,2M,322.0,NaN,NaN
4,20220327BZECYC2M202203,20220327,202203,BZECYC,CAMCAM,OTHS,2M,322.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2916,20230319OAKSALY4202303,20230319,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0,NaN,NaN
2917,20230326OAKSALY4202303,20230326,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0,NaN,NaN
2918,20230507BLBUIO~~202305,20230507,202305,BLBUIO,CAMSAM,ECUINT,~~,1606515.0,minimo,NaN
2919,20230507ETRUIO~~202305,20230507,202305,ETRUIO,DOM_EC,ECUDOM,~~,2457782.0,minimo,NaN


In [52]:
salida_02['etiqueta_01'] = salida_02.apply(lambda row: 'ON' if row['minimo']=='minimo' or row['Cuenta_ceros_ventana']==3 else 'OFF', axis=1)
salida_02['etiqueta_02'] = salida_02.apply(lambda row: row['snapshot_date'] if row['etiqueta_01']=='ON' else 'OFF', axis=1)
salida_02

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks,minimo,Cuenta_ceros_ventana,etiqueta_01,etiqueta_02
0,20220213BZECUK2M202201,20220213,202201,BZECUK,CAMCAM,OTHS,2M,103272.0,minimo,NaN,ON,20220213
1,20220227BZECUK2M202201,20220227,202201,BZECUK,CAMCAM,OTHS,2M,103272.0,NaN,NaN,OFF,OFF
2,20220227BZECYC2M202203,20220227,202203,BZECYC,CAMCAM,OTHS,2M,322.0,minimo,NaN,ON,20220227
3,20220313BZECYC2M202203,20220313,202203,BZECYC,CAMCAM,OTHS,2M,322.0,NaN,NaN,OFF,OFF
4,20220327BZECYC2M202203,20220327,202203,BZECYC,CAMCAM,OTHS,2M,322.0,NaN,NaN,OFF,OFF
...,...,...,...,...,...,...,...,...,...,...,...,...
2916,20230319OAKSALY4202303,20230319,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0,NaN,NaN,OFF,OFF
2917,20230326OAKSALY4202303,20230326,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0,NaN,NaN,OFF,OFF
2918,20230507BLBUIO~~202305,20230507,202305,BLBUIO,CAMSAM,ECUINT,~~,1606515.0,minimo,NaN,ON,20230507
2919,20230507ETRUIO~~202305,20230507,202305,ETRUIO,DOM_EC,ECUDOM,~~,2457782.0,minimo,NaN,ON,20230507


In [53]:
salida_03 = salida_02.drop(['minimo',	'Cuenta_ceros_ventana'], axis=1)
salida_03

,id_alert,snapshot_date,flight_date,rt,region,hub,airline_code,asks,etiqueta_01,etiqueta_02
0,20220213BZECUK2M202201,20220213,202201,BZECUK,CAMCAM,OTHS,2M,103272.0,ON,20220213
1,20220227BZECUK2M202201,20220227,202201,BZECUK,CAMCAM,OTHS,2M,103272.0,OFF,OFF
2,20220227BZECYC2M202203,20220227,202203,BZECYC,CAMCAM,OTHS,2M,322.0,ON,20220227
3,20220313BZECYC2M202203,20220313,202203,BZECYC,CAMCAM,OTHS,2M,322.0,OFF,OFF
4,20220327BZECYC2M202203,20220327,202203,BZECYC,CAMCAM,OTHS,2M,322.0,OFF,OFF
...,...,...,...,...,...,...,...,...,...,...
2916,20230319OAKSALY4202303,20230319,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0,OFF,OFF
2917,20230326OAKSALY4202303,20230326,202303,OAKSAL,CAMNAM,SAL,Y4,6312439.0,OFF,OFF
2918,20230507BLBUIO~~202305,20230507,202305,BLBUIO,CAMSAM,ECUINT,~~,1606515.0,ON,20230507
2919,20230507ETRUIO~~202305,20230507,202305,ETRUIO,DOM_EC,ECUDOM,~~,2457782.0,ON,20230507


In [ ]:
df[(df.flight_date=='202201')&
   (df.airline_code=='2M')&
   (df.rt=='BZECUK')
]